In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from causalml.match import NearestNeighborMatch
from causalml.inference.meta import BaseXRegressor
from sklearn.ensemble import RandomForestClassifier

# Data Preparation

In [ ]:
# Load the dataset
data = pd.read_csv('ards_data.csv')

# Assuming the dataset has columns for patient demographics, ARDS severity,
# sedation details, and a binary outcome for delirium (1 for presence, 0 for absence)
print(data.head())

# Calculating Propensity Scores

In [ ]:
# Define the treatment and outcome variables
treatment = 'sedation_type'  # This is your categorical variable for the type of sedation
outcome = 'delirium'

# Define confounders
confounders = ['age', 'sex', 'ards_severity', 'comorbidities']

# Convert categorical treatment to dummy variables
treatment_dummies = pd.get_dummies(data[treatment])
confounder_variables = data[confounders]

# Fit a logistic regression model for each treatment type
propensity_scores = pd.DataFrame()
for treatment_type in treatment_dummies.columns:
    model = LogisticRegression()
    model.fit(confounder_variables, treatment_dummies[treatment_type])
    # The predicted probabilities are your propensity scores
    propensity_scores[treatment_type] = model.predict_proba(confounder_variables)[:, 1]


# Propensity Score Matching

In [ ]:
# Initialize the matching method
nnm = NearestNeighborMatch(replace=False, ratio=1, random_state=42)

# Choose a treatment type to focus on, e.g., Propofol
treatment_focus = 'Propofol'

# Perform matching
matched_indices = nnm.match(data=data, treatment=treatment_dummies[treatment_focus], score=propensity_scores[treatment_focus])

# Extract matched data
matched_data = data.iloc[matched_indices]

# Analyze the matched data
# For example, compare the incidence of delirium between groups
model = BaseXRegressor(learner=RandomForestClassifier())
ate = model.estimate_ate(X=matched_data[confounders], treatment=matched_data[treatment_focus], y=matched_data[outcome])
print(f"Average Treatment Effect (ATE) of {treatment_focus} on delirium: {ate[0]}")
